# Plot graphs
Plot some graphs (pre-deformation case, affine case, final deformation) for illustrative purposes.

In [ ]:
import pickle
import matplotlib.pyplot as plt
import numpy as np
import torch

In [ ]:
with open("data\coarseMesh\graphs_coarseMesh_diameter_0.9_alts7.pkl", 'rb') as f:
# with open(r"data\coarseMesh_noBifurcation_5\graphs_coarseMesh_noBifurcation_diameter_0.9_5_noBulkNodes_2.pkl", 'rb') as f:
    data_list0 = pickle.load(f)

print(data_list0[0])
print(len(data_list0))

In [ ]:
F = np.concatenate([graph.F for graph in data_list0], axis=0)
F.shape

In [ ]:
# Find an F that has some biaxial compression (10-20%) and some shear (also 10-20%)
inds = np.where((F[:, 0, 0] < 0.9)
                *(F[:, 0, 0] > 0.8)
                *(F[:, 1, 1] < 0.9)
                *(F[:, 1, 1] > 0.8)
                *(F[:, 0, 1] > 0.1)
                *(F[:, 0, 1] < 0.2))[0]
print(inds)
ind = inds[0]
print(F[ind])

print(f'{np.abs(F[ind][0,0]-1)*100:.2f}% {"compression" if F[ind][0,0]-1 < 0 else "tension"} in x-direction')
print(f'{np.abs(F[ind][1,1]-1)*100:.2f}% {"compression" if F[ind][0,0]-1 < 0 else "tension"} in y-direction')
print(f'{np.abs(F[ind][0,1])*100:.2f}% shear')

In [ ]:
graph = data_list0[inds[0]]

pos1 = graph.pos
pos2 = torch.matmul(graph.F[0], graph.pos.T).T
pos3 = graph.y[..., 0]

In [ ]:
plot_affine = False
plot_reference = False
plot_final = True

In [ ]:
%matplotlib qt

fig, ax = plt.subplots()
fig.patch.set_facecolor("None")

x, y = np.transpose(pos1[graph.edge_index], axes=[2,0,1])
# filter out wraparound edges
bools = ((np.abs(np.diff(x, axis=0)) < 0.5)
            & (np.abs(np.diff(y, axis=0)) < 0.5)
        ).flatten()

if plot_reference:
    # plot original positions
    ax.scatter(*pos1.T, label='original position', s=1, c='tab:red')
    edges0 = ax.plot(x[:, bools], y[:, bools], alpha=0.3, c='tab:red')

if plot_affine:
    # plot affine positions
    ax.scatter(*pos2.T, label='new position', s=1, c='tab:blue')
    x, y = np.transpose(pos2[graph.edge_index], axes=[2,0,1])
    edges0 = ax.plot(x[:, bools], y[:, bools], alpha=0.3, c='tab:blue')

if plot_final:
    # plot new positions
    ax.scatter(*pos3.T, label='new position', s=1, c='tab:green')
    x, y = np.transpose(pos3[graph.edge_index], axes=[2,0,1])
    edges0 = ax.plot(x[:, bools], y[:, bools], alpha=0.3, c='tab:green')

ax.set_xlim([-2.0, 2.0])
ax.set_ylim([-2.0, 2.0])

# make plots pretty
ax.set_aspect('equal')
# plt.grid()
ax.axis('off')

ax.margins(0)

plt.tight_layout()

path = f'results/graph{"_reference"*plot_reference}{"_affine"*plot_affine}{"_final"*plot_final}_F=[{F[ind][0]},{F[ind][1]}].svg'
plt.gcf().savefig(path)

# Plot graph without bulk nodes

In [1]:
import pickle
import matplotlib.pyplot as plt
import numpy as np
import torch

In [3]:
with open(r"data\coarseMesh_noBifurcation_5\graphs_coarseMesh_noBifurcation_diameter_0.9_5_noBulkNodes_2.pkl", 'rb') as f:
    data_list0 = pickle.load(f)

print(data_list0[0])
print(len(data_list0))
graph = data_list0[0]

Data(x=[128, 0], edge_index=[2, 960], edge_attr=[960, 1], y=[128, 2], pos=[128, 2], r=[960, 2], d=[960, 1], mean_pos=[1, 2], W=[1], P=[1, 2, 2], D=[1, 2, 2, 2, 2], F=[1, 2, 2], traj=[1])
8451


In [36]:
%matplotlib qt

# plot all nodes in original location
# plt.scatter(*(n_data['pos']), c='black', s=1)  #, s=50, alpha=0.5)

# plot nodes on hole bound
plt.scatter(*graph.pos.T, s=10)  #, marker='x', s=50, c=quad)

# plot edges
pos1 = graph.pos[graph.edge_index[0][:480]]
pos2 = pos1 + graph.r[:480]
x = np.stack((pos1[:, 0], pos2[:, 0]))
y = np.stack((pos1[:, 1], pos2[:, 1]))
plt.plot(x, y, c='red', alpha=0.3, zorder=-1)

# plot boundary edges
b_edge_index = graph.edge_attr[:, 0][:480] == -1
pos1 = graph.pos[graph.edge_index[:, :480][0, b_edge_index]]
pos2 = pos1 + graph.r[:480][b_edge_index]
x = np.stack((pos1[:, 0], pos2[:, 0]))
y = np.stack((pos1[:, 1], pos2[:, 1]))
plt.plot(x, y, c='green', zorder=-1)

# make plot pretty
plt.gca().set_aspect('equal')
plt.xlabel('x')
plt.ylabel('y')
plt.grid()

# Plot geometry

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# 10×10 unit cell grid
%matplotlib qt
fig = plt.figure()

corners = np.array([[0, 0.0], [0.0, 1], [1., 1.], [1, 0]])
mid_point = np.array([0.5, 0.5])
r = 0.9/2

for i in range(10):
    for j in range(10):
        shift = np.array([i, j])
        if 3 <= i <= 4 and 3<=j <= 4:
            plt.fill(*(corners + shift).T, c='tab:blue')
        else:
            plt.fill(*(corners + shift).T, c='tab:orange')

        circle1 = plt.Circle((mid_point+shift), r, color='white')


        plt.gca().add_artist(circle1)

plt.gca().set_aspect('equal')

In [ ]:
# 2×2 unit cell grid
%matplotlib qt
fig = plt.figure()

corners = np.array([[0, 0.0], [0.0, 1], [1., 1.], [1, 0]])
mid_point = np.array([0.5, 0.5])
r = 0.85/2

for i in range(2):
    for j in range(2):
        shift = np.array([i, j])
        plt.fill(*(corners + shift).T, c='tab:blue')

        circle1 = plt.Circle((mid_point+shift), r, color='white')


        plt.gca().add_artist(circle1)

ax = plt.gca()
# Turn off the axes
ax.axis('off')

# Make the background transparent
ax.patch.set_alpha(0)
plt.gca().set_aspect('equal')
plt.savefig('geometry.svg')